In [3]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import pandas as pd
import sys
import datetime
from pathlib import Path
sys.path.append("../")
sys.path.append("../../")

sys.path.append("./analysis")
sys.path.append("./analysis/technical-analysis_python")
mpl.use('tkagg')  # issues with Big Sur

import matplotlib.pyplot as plt
from strategy.macd_crossover import macdCrossover
from backtest import Backtest
from evaluate import PortfolioReturn, SharpeRatio, MaxDrawdown, CAGR

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
directory_name = "hkex_ticks_day"

pathlist = Path(directory_name).rglob('*.csv')

# traverse over files in directory
for path in pathlist:
    
    # load data
    df = pd.read_csv(path, header=0, index_col='Date', parse_dates=True)
    
    # select time range
    df = df.loc[pd.Timestamp('2017-01-01'):pd.Timestamp('2019-01-01')]
    
    path_str = str(path)
    ticker = path_str[len(directory_name)+6:len(directory_name)+6+4]
    
    # MACD
    """
    macd_cross = macdCrossover(df)
    macd_fig = macd_cross.plot_MACD()
    mpl.pyplot.close()
    
    signals = macd_cross.gen_signals()
    signal_fig = macd_cross.plot_signals(signals)
    mpl.pyplot.close()
    
    signal_fig = macd_cross.plot_signals_MACD()
    mpl.pyplot.close()
    """
    
    # RSI

    rsi = rsi(df)
    rsi_fig = rsi.plot_RSI()
    mpl.pyplot.close()
    
    signals = rsi.gen_signals()
    signal_fig = rsi.plot_signals(signals)
    mpl.pyplot.close()
    
    # Backtesting
    portfolio, backtest_fig = Backtest(ticker, signals, df)
    mpl.pyplot.close()
    
    
    if (len(portfolio) != 0):
        print("Final total value: {value:.4f} ".format(value = portfolio['total'][-1]))
        print("Total return: {value:.4f}%".format(value = (portfolio['total'][-1] - portfolio['total'][0])/portfolio['total'][-1]*100))
        # for analysis
        print("No. of trade: {value}".format(value = len(signals[signals.positions == 1])))
        
        filename = "diff-tickers-2-years-RSI.txt"
        
        with open(filename,'a') as outfile:
            port_ret = (portfolio['total'][-1] - portfolio['total'][0])/portfolio['total'][-1]*100
            num_trade = len(signals[signals.positions == 1])
            start_date = pd.Timestamp('2017-01-01').strftime("%Y-%m-%d")
            end_date = pd.Timestamp('2019-06-01').strftime("%Y-%m-%d")
            
            outfile.write("{},{},{},{},{}".format(ticker, start_date, end_date, port_ret, num_trade))
            outfile.write('\n')